In [ ]:
import innvestigate
import innvestigate.utils

# Strip softmax layer
model2_nsm = innvestigate.utils.model_wo_softmax(model2)

# Create analyzer
#analyzer = innvestigate.create_analyzer("lrp.", model2_nsm)
analyzer = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPEpsilonIgnoreBias(model2_nsm)

a = np.zeros((101,224,224))
print(a.shape)

#split the job and analyze all 101 images
N=10
L=N
for i in range(11):
    #shrink the last job to fit
    if (((i*N)+L)>101):
        L = 101 - i*N

    print('analyzing images',i*N,"to",i*N+L,"...")
    image=batch[i*N:i*N+L]

    # Apply analyzer w.r.t. maximum activated output-neuron
    an = analyzer.analyze(image)

    # Aggregate along color channels and normalize to [-1, 1]
    print(1,a[i*N:i*N+L].shape)
    a[i*N:i*N+L] = an.sum(axis=np.argmax(np.asarray(an.shape) == 3))
    a[i*N:i*N+L] /= np.max(np.abs(a[i*N:i*N+L]))

In [ ]:
# Plot
for i in range (len(batch)):
    plt.figure(figsize=(10, 4))
    plt.subplot(1,4,1)
    plt.imshow(a[i], cmap="seismic", clim=(-1, 1))
    plt.subplot(1,4,2)
    plt.imshow(batch[i]/255 + 0.5)
    plt.subplot(1,4,3)
    a_filt = scipy.ndimage.maximum_filter(a[i],footprint=np.ones((3,3)))
    plt.imshow(a_filt, cmap="seismic", clim=[-1,1])
    plt.subplot(1,4,4)
    a_filt2 = scipy.ndimage.minimum_filter(a[i],footprint=np.ones((3,3)))
    plt.imshow(a_filt2, cmap="seismic", clim=[-1,1])

Consider statistics

In [ ]:
total, eyes, nose, mouth, brow, outside, inside = [], [], [], [] ,[], [], [];
plt.figure(figsize=(20,10))
plt.subplots_adjust(hspace = 0.4)
for i in range (len(batch)):
    #print(front_view_filenames[i])
    total.append(np.sum(np.abs(a[i])))
    #print("total relevance:",total[i])
    eyes.append(np.sum(np.abs(a[i][100:130,60:160])) / total[i])
    #print("eyes relevance:",eyes[i])
    nose.append(np.sum(np.abs(a[i][120:160,90:130])) / total[i])
    #print("nose relevance:",nose[i])
    mouth.append(np.sum(np.abs(a[i][160:190,75:145])) / total[i])
    #print("mouth relevance:",mouth[i])
    brow.append(np.sum(np.abs(a[i][70:100,50:170])) / total[i])
    #print("brow relevance:",brow[i])
    
    outside.append(1-(eyes[i]+nose[i]+mouth[i]+brow[i]))
    inside.append((eyes[i]+nose[i]+mouth[i]+brow[i]))
    
    if i <= 4:
        plt.subplot(2,3,i+1)
        plt.title("Relevance of features, Identity "+front_view_filenames[i].split("/")[3])
        plt.bar(x=[ "eyes", "nose", "mouth", "brow"],height=[ eyes[i]*100, nose[i]*100, mouth[i]*100, brow[i]*100])
        plt.grid(axis='y')
        plt.xlabel('face region')
        plt.ylabel('% of total absolute relevance')
    
plt.subplot(2,3,6)
plt.title("Relevance of features, Average (N=101)")
plt.bar(x=[ "eyes", "nose", "mouth", "brow"],
        height=[ np.mean(eyes)*100,  np.mean(nose)*100,  np.mean(mouth)*100,  np.mean(brow)*100],
        yerr = [ np.std(eyes)*100,  np.std(nose)*100,  np.std(mouth)*100,  np.std(brow)*100])
plt.grid(axis='y')
plt.xlabel('face region')
plt.ylabel('% of total absolute relevance');

In [ ]:
plt.figure()
plt.bar(x=[ "internal", "external"],
        height=[ np.mean(inside)*100,  np.mean(outside)*100],
        yerr = [ np.std(inside)*100,  np.std(outside)*100])
plt.grid(axis='y')
plt.xlabel('face region')
plt.ylabel('% of total absolute relevance');
plt.title("Relevance of internal vs. external features")

In [ ]:
avg_face_forward = np.mean(batch, axis=(0))
avg_rel_forward = np.mean(a, axis=(0))
avg_rel_forward /= np.max(np.abs(avg_rel_forward))

plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.imshow(avg_rel_forward, cmap="seismic", clim=[-1,1])
plt.title('Average relevance map (N=101)')
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(avg_face_forward/255 + 0.5)
plt.title("Average face image (N=101)")

In [ ]:
masks = []
features = ["cheeks","hair","chin","mouth","nose","brows","eyes"]
results = []
pixels = []
densities = []

total = np.sum(np.abs(avg_rel_forward))
for i in range(len(features)):
    masks.append(np.mean(np.array(image_utils.load_img("./masks/mask_"+features[i]+".png",target_size=(224, 224)))/255.0,axis=-1))
    plt.subplot(1,7,i+1)
    plt.imshow(masks[i])
    plt.title(features[i])

    pixels.append(np.sum(masks[i]))
    results.append(np.sum(np.abs(masks[i]*avg_rel_forward)/total))
    densities.append(results[i]/pixels[i])
    
plt.figure()
plt.bar(x=features,
        height=results)
plt.grid(axis='y')
plt.xlabel('face region')
plt.ylabel('% of total absolute relevance');
plt.title("Relevance of internal features in averaged face (N=101)")

In [ ]:
plt.figure()
plt.bar(x=features,
        height=densities)
plt.grid(axis='y')
plt.xlabel('face region')
plt.ylabel('% of total absolute relevance per pixel');
plt.title("Relevance \"density\" of internal features in averaged face (N=101)")